In [1]:
from movie_recommender import MovieRecommender
import timeit

found pickle at c:\Users\john.french\movie_recs\data\pickled\movie_ratings.pkl
  (0, 120)	5.0
  (0, 183)	5.0
  (0, 228)	5.0
  (0, 289)	5.0
  (0, 313)	5.0
  (0, 325)	5.0
  (0, 351)	5.0
  (0, 352)	5.0
  (0, 358)	5.0
  (0, 360)	5.0
  (0, 366)	5.0
  (0, 373)	5.0
  (0, 416)	5.0
  (0, 462)	5.0
  (0, 476)	5.0
  (0, 516)	5.0
  (0, 535)	5.0
  (0, 580)	5.0
  (0, 582)	5.0
  (0, 583)	5.0
  (0, 588)	5.0
  (0, 610)	5.0
  (1, 108)	5.0
  (1, 149)	3.0
  (1, 257)	5.0
  :	:
  (69877, 1546)	2.0
  (69877, 1633)	3.0
  (69877, 1657)	1.0
  (69877, 1661)	4.0
  (69877, 1684)	3.0
  (69877, 1700)	3.0
  (69877, 1719)	2.0
  (69877, 1729)	4.0
  (69877, 1752)	3.0
  (69877, 1792)	3.0
  (69877, 1825)	2.0
  (69877, 1833)	4.0
  (69877, 1836)	4.0
  (69877, 1898)	1.0
  (69877, 1899)	1.0
  (69877, 1900)	1.0
  (69877, 1901)	1.0
  (69877, 1902)	1.0
  (69877, 1928)	3.0
  (69877, 1944)	5.0
  (69877, 2023)	1.0
  (69877, 2042)	2.0
  (69877, 2210)	5.0
  (69877, 2254)	2.0
  (69877, 2300)	2.0
Top 5 users who are very much similar to

In [2]:
started = timeit.default_timer()
movies_file = 'data/ml-10M100K/movies.dat'
ratings_file = 'data/ml-10M100K/ratings.dat'
movie_rec = MovieRecommender(movies_filename=movies_file, ratings_filename=ratings_file)
elapsed = timeit.default_timer() - started
print(f'Read dataframes in {elapsed}ms')

print(movie_rec.ratings_df['user_id'].nunique())

found pickle at c:\Users\john.french\movie_recs\data\pickled\movie_ratings.pkl
  (0, 120)	5.0
  (0, 183)	5.0
  (0, 228)	5.0
  (0, 289)	5.0
  (0, 313)	5.0
  (0, 325)	5.0
  (0, 351)	5.0
  (0, 352)	5.0
  (0, 358)	5.0
  (0, 360)	5.0
  (0, 366)	5.0
  (0, 373)	5.0
  (0, 416)	5.0
  (0, 462)	5.0
  (0, 476)	5.0
  (0, 516)	5.0
  (0, 535)	5.0
  (0, 580)	5.0
  (0, 582)	5.0
  (0, 583)	5.0
  (0, 588)	5.0
  (0, 610)	5.0
  (1, 108)	5.0
  (1, 149)	3.0
  (1, 257)	5.0
  :	:
  (69877, 1546)	2.0
  (69877, 1633)	3.0
  (69877, 1657)	1.0
  (69877, 1661)	4.0
  (69877, 1684)	3.0
  (69877, 1700)	3.0
  (69877, 1719)	2.0
  (69877, 1729)	4.0
  (69877, 1752)	3.0
  (69877, 1792)	3.0
  (69877, 1825)	2.0
  (69877, 1833)	4.0
  (69877, 1836)	4.0
  (69877, 1898)	1.0
  (69877, 1899)	1.0
  (69877, 1900)	1.0
  (69877, 1901)	1.0
  (69877, 1902)	1.0
  (69877, 1928)	3.0
  (69877, 1944)	5.0
  (69877, 2023)	1.0
  (69877, 2042)	2.0
  (69877, 2210)	5.0
  (69877, 2254)	2.0
  (69877, 2300)	2.0
Read dataframes in 38.340137699851766ms


In [9]:
u = 2
print(f'User {u} has seen...')
for m in movie_rec.get_user_movies(u):
    print(movie_rec.get_movie_title(m))

User 2 has seen...
Star Wars: Episode IV - A New Hope (a.k.a. Star Wars)
Mission: Impossible
Lost World: Jurassic Park, The (Jurassic Park 2)
Independence Day (a.k.a. ID4)
Eraser
Rob Roy
Ghost and the Darkness, The
Sleepless in Seattle
Phenomenon
Willy Wonka & the Chocolate Factory
Star Wars: Episode VI - Return of the Jedi
Star Trek: First Contact
Dances with Wolves
Multiplicity
Godfather, The
Rock, The
Twister
Braveheart
Mars Attacks!
River Wild, The


In [10]:
# rec = movie_rec.get_naive_recommendation(user_id)
movie_rec.get_user_based_recommendation(u)

Top 5 users who are very much similar to the User- 2 are: 
 
1 . User: 48761 separated by distance of 0.4456551862194902
2 . User: 59801 separated by distance of 0.4637693686077883
3 . User: 46549 separated by distance of 0.4822268317078031
4 . User: 38590 separated by distance of 0.4899446008946575
5 . User: 42710 separated by distance of 0.49034163221142435
[0.18788655 0.19552343 0.20330502 0.20655881 0.20672619]
[0. 0. 0. ... 0. 0. 0.]
American Beauty
Schindler's List
Forrest Gump


In [7]:
users, dists = movie_rec.get_similar_users(u)
# for rec_user in users:
#     # print(f'{rec_user}\'s movies')
#     for m in movie_rec.get_user_movies(rec_user):
#         # print(movie_rec.get_movie_title(m))

# print(movie_rec.get_movie_title(20))
# print(movie_rec.get_movie_title(25))
# print(movie_rec.get_movie_title(50))

Top 5 users who are very much similar to the User- 1 are: 
 
1 . User: 25697 separated by distance of 0.4093584617331092
2 . User: 20577 separated by distance of 0.44565560596361065
3 . User: 6156 separated by distance of 0.46359176245775724
4 . User: 25060 separated by distance of 0.4751821967780011
5 . User: 61323 separated by distance of 0.4801513497976386


In [3]:
user_id = movie_rec.get_next_avail_user_id()
# movie_rec.add_movie_rating(movie_id=1, user_id= user_id, rating= 5) # Toy Story
# movie_rec.add_movie_rating(movie_id=2, user_id= user_id, rating= 5) # Jumanji
# movie_rec.add_movie_rating(movie_id=48, user_id= user_id, rating= 5) # Pocahontas
movie_rec.add_movie_rating(movie_id=20, user_id= user_id, rating= 5) # Money Train
movie_rec.add_movie_rating(movie_id=25, user_id= user_id, rating= 5) # Leaving Las Vegas
movie_rec.add_movie_rating(movie_id=50, user_id= user_id, rating= 5) # The Usual Suspects
movie_rec.ratings_df.head()

,user_id,rating,timestamp
movie_id,,,
122,1,5,838985046.0
185,1,5,838983525.0
231,1,5,838983392.0
292,1,5,838983421.0
316,1,5,838983392.0


In [ ]:
import numpy as np


# return the difference between average ratings for two movies
def get_movie_mean_rating_error(movie1_id: int, movie2_id: int):
    movie1_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie1_id]['rating'].values)
    movie2_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie2_id]['rating'].values)
    return abs(movie1_ratings - movie2_ratings)


# get_movie_mean_rating_error(599, 700)
# movie_rec.get_movie_users(5)
print(movie_rec.get_movie_title(1))
print(movie_rec.get_movie_title(2))
print(len(movie_rec.get_movie_users(1)))
print(len(movie_rec.get_movie_users(2)))
print(len(movie_rec.get_common_users(1, 2)))

In [ ]:
def get_movie_rating_pearson_coeff(movie1_id: int, movie2_id: int)-> float:
    common_users = movie_rec.get_common_users(movie1_id=movie1_id, movie2_id=movie2_id)
    movie1_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie1_id) for u in common_users]
    movie2_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie2_id) for u in common_users]
    coeff = np.corrcoef(np.array(movie1_ratings), np.array(movie2_ratings))[0, 1]
    movie1_title = movie_rec.get_movie_title(movie1_id)
    movie2_title = movie_rec.get_movie_title(movie2_id)
    print(f'{movie1_title} and {movie2_title} have a correlation of {"{:.2f}".format(coeff)} from {len(common_users)} users who have rated both')

get_movie_rating_pearson_coeff(1, 3)